In [1]:
import tensorflow  as tf 
import pathlib
import numpy as np 
import os 

2023-09-30 14:00:04.117175: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-30 14:00:04.775690: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def convert_bytes(num):
    """
    this function will convert bytes to MB.... GB... etc
    """
    for x in ['bytes', 'KB', 'MB', 'GB', 'TB']:
        if num < 1024.0:
            return "%3.1f %s" % (num, x)
        num /= 1024.0


def file_size(file_path):
    """
    this function will return the file size
    """
    if os.path.isfile(file_path):
        file_info = os.stat(file_path)
        return convert_bytes(file_info.st_size)

In [83]:
model = tf.keras.models.load_model('D:/Github/TF Quantization/yolov8l.tf')

In [4]:
# Lets check the file size of MS Paint exe 
# or you can use any file path
file_path = r"/home/airi/yolo/ONNX-TensorRT-Pytorch-Tensorflow-Face-Detection-Models-Quantization/models/yolov8l.tf/saved_model.pb"
print(file_size(file_path))

167.5 MB


In [66]:
PATH = 'D:/Github/TF Quantization/yolov8l.tf'

### Dynamic range quantization
Dynamic range quantization is a recommended starting point because it provides reduced memory usage and faster computation without you having to provide a representative dataset for calibration. 

In [4]:
converter = tf.lite.TFLiteConverter.from_saved_model(PATH)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()

In [5]:
open("D:/Github/TF Quantization/quantization_DEFAULT.tflite", "wb").write(tflite_quant_model)

44178952

To further reduce latency during inference, "dynamic-range" operators dynamically quantize activations based on their range to 8-bits and perform computations with 8-bit weights and activations. This optimization provides latencies close to fully fixed-point inferences. However, the outputs are still stored using floating point so the increased speed of dynamic-range ops is less than a full fixed-point computation.

In [10]:
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="D:/Github/TF Quantization/quantization_DEFAULT.tflite")
interpreter.allocate_tensors()

In [12]:
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
# Get model weights
model_weights = interpreter.get_tensor_details()

In [64]:
# Example: Print the weights of the first tensor
tensor_name = model_weights[0]['index']
tensor_weights = interpreter.get_tensor(tensor_name)
print(tensor_weights[0])
print(tensor_weights[0][0][0][0].dtype)

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
float32


In [5]:
# Quantized model size
file_path = r"/home/airi/yolo/ONNX-TensorRT-Pytorch-Tensorflow-Face-Detection-Models-Quantization/quant_models/quantization_DEFAULT.tflite"
print(file_size(file_path))

42.1 MB
